In [ ]:
# | default_exp export_step

# Export cells with a named step

> Exporting `nbdev` notebook cells with a name adds the ability to build name pattern based differentiation of downstream activity.

In [ ]:
# | export

import os
from collections import defaultdict
from warnings import warn

import nbdev
from fastcore.basics import ifnone
from fastcore.foundation import L
from fastcore.meta import delegates
from fastcore.script import Param, call_parse
from nbdev.config import get_config
from nbdev.doclinks import _build_modidx, add_init, nbglob, nbglob_cli
from nbdev.maker import ModuleMaker
from nbdev.process import NBProcessor

In [ ]:
# |export
# adapted from nbdev.export


class ExportModuleProc:
    "A processor which exports code to a module"

    def begin(self):
        self.modules, self.in_all = defaultdict(L), defaultdict(L)

    def _default_exp_(self, cell, exp_to):
        self.default_exp = exp_to

    def _exporti_(self, cell, exp_to=None):
        self.modules[ifnone(exp_to, "#")].append(cell)

    def _export_(self, cell, exp_to=None):
        self._exporti_(cell, exp_to)
        self.in_all[ifnone(exp_to, "#")].append(cell)

    # Add a named step directive

    def _export_step_(self, cell, step_name, exp_to=None):
        self._exporti_(cell, exp_to)
        self.in_all[ifnone(exp_to, "#")].append(cell)

    _exports_ = _export_

patch export to use new proc

In [ ]:
# |export
# copied from nbdv.export


def optional_procs():
    "An explicit list of processors that could be used by `nb_export`"
    return L(
        [
            p
            for p in nbdev.export.__all__
            if p not in ["nb_export", "ExportModuleProc", "optional_procs"]
        ]
    )

In [ ]:
# |export
# adapted from nbdev.export


def nb_named_export(
    nbname, lib_path=None, procs=None, debug=False, mod_maker=ModuleMaker, name=None
):
    "Create module(s) from notebook"
    if lib_path is None:
        lib_path = get_config().lib_path
    exp = ExportModuleProc()
    # Changed default value of remove directives
    nb = NBProcessor(nbname, [exp] + L(procs), debug=debug, rm_directives=False)
    nb.process()
    for mod, cells in exp.modules.items():
        all_cells = exp.in_all[mod]
        nm = ifnone(name, getattr(exp, "default_exp", None) if mod == "#" else mod)
        if not nm:
            warn(
                f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
                "Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.\n"
                "See https://nbdev.fast.ai/getting_started.html for more information."
            )
            return
        mm = mod_maker(
            dest=lib_path, name=nm, nb_path=nbname, is_new=bool(name) or mod == "#"
        )
        mm.make(cells, all_cells, lib_path=lib_path)

In [ ]:
# |export
# adapted from nbdev.doclinks


@call_parse
@delegates(nbglob_cli)
def sciflow_export(
    path: str = None,  # Path or filename
    procs: Param(
        "tokens naming the export processors to use.",
        nargs="*",
        choices=optional_procs(),
    ) = "black_format",
    **kwargs,
):
    "Export notebooks in `path` to Python modules"
    if os.environ.get("IN_TEST", 0):
        return
    if procs:
        import nbdev.export

        procs = [getattr(nbdev.export, p) for p in L(procs)]
    files = nbglob(path=path, as_path=True, **kwargs).sorted("name")
    for f in files:
        nb_named_export(f, procs=procs)
    add_init(get_config().lib_path)
    _build_modidx()

In [ ]:
path: str = None
files = nbglob(path=path, as_path=True).sorted("name")

for f in files:
    nb_named_export(f, procs=None)
    add_init(get_config().lib_path)
    _build_modidx()